In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model

import datetime

import numpy as np

from plot import vessel_planning

In [2]:
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic


In [3]:
# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(5.1, 52)  # lon, lat
data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie1",  # The name of the site
    "geometry": location_from_site,  # The coordinates of the project site
    "capacity": 24,
    "level": 24,
    "nr_resources":3
}  # The actual volume of the site
from_site = Site(**data_from_site)


location_to_site = shapely.geometry.Point(5, 52)  # lon, lat
data_to_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Dumplocatie",  # The name of the site
    "geometry": location_to_site,  # The coordinates of the project site
    "capacity": 24,
    "level": 0,
}  # The actual volume of the site (empty of course)
to_site = Site(**data_to_site)


# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.ContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.HasCosts,  # Add information on costs
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
        # SiteRegistry,
    ),
    {"key": "MultiStoreHopper"},
)

 
data_cutter1 = {
    "env": my_env,  # The simpy environment
    "name": "Cutter1",  # Name
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 1,
    "level": 0,
    "compute_v": lambda x: 10,  # Variable speed
    "weekrate": 7,
}
cutter1 = TransportProcessingResource(**data_cutter1)
data_cutter2 = {
    "env": my_env,  # The simpy environment
    "name": "Cutter2",  # Name
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 1,
    "level": 0,
    "compute_v": lambda x: 10,  # Variable speed
    "weekrate": 7,
}
cutter2 = TransportProcessingResource(**data_cutter2)
 

# TSHD variables
data_hopper1 = {
    "env": my_env,  # The simpy environment
    "name": "Hopper1",  # Name
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 1,
    "level": 0,
    "compute_v": lambda x: 10,  # Variable speed
    "weekrate": 7,
}
hopper1 = TransportProcessingResource(**data_hopper1)
data_hopper2 = {
    "env": my_env,  # The simpy environment
    "name": "Hopper2",  # Name
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 1,
    "level": 0,
    "compute_v": lambda x: 10,  # Variable speed
    "weekrate": 7,
}
hopper2 = TransportProcessingResource(**data_hopper2)
data_hopper3 = {
    "env": my_env,  # The simpy environment
    "name": "Hopper3",  # Name
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 1,
    "level": 0,
    "compute_v": lambda x: 10,  # Variable speed
    "weekrate": 7,
}
hopper3 = TransportProcessingResource(**data_hopper3)

init
level: 24
completed init
init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init


In [4]:
# definition of loading a barge
registry = {}
cutter_list = [cutter1, cutter2]
for hopper in [hopper1, hopper2, hopper3]:
    first_cutter = cutter_list[0]
    cutter_list= cutter_list[1:]
    cutter_list.append(first_cutter)
    for cutter in cutter_list:
        requested_resources = {}
        run = []
        
        move_activity_to_harbor_data = {
            "env": my_env,  # The simpy environment defined in the first cel
            "name": "sailing empty",  # We are moving soil
            "registry": registry,
            "mover": hopper,
            "destination": from_site,
            "requested_resources":requested_resources,
            "postpone_start": True,
#             "keep_resources":[hopper],
        }
        run.append(model.MoveActivity(**move_activity_to_harbor_data))
        
        shift_amount_loading_data = {
            "env": my_env,  # The simpy environment defined in the first cel
            "name": "Load",  # We are moving soil
            "registry": registry,
            "processor": cutter,
            "origin": from_site,
            "destination": hopper,
            "amount": 1,
            "duration": 3600,
            "requested_resources":requested_resources,
#             "keep_resources":[hopper],
            "postpone_start": True,
        }
        run.append(model.ShiftAmountActivity(**shift_amount_loading_data))
        
        move_activity_to_harbor_data = {
            "env": my_env,  # The simpy environment defined in the first cel
            "name": "sailing full",  # We are moving soil
            "registry": registry,
            "mover": hopper,
            "destination": to_site,
            "requested_resources":requested_resources,
#             "keep_resources":[hopper],
            "postpone_start": True,
        }
        run.append(model.MoveActivity(**move_activity_to_harbor_data))

 
        shift_amount_loading_data = {
            "env": my_env,  # The simpy environment defined in the first cel
            "name": "Unload",  # We are moving soil
            "registry": registry,
            "processor": hopper,
            "origin": hopper,
            "destination": to_site,
            "amount": 1,
            "duration": 3600,
            "requested_resources":requested_resources,
#             "keep_resources":[hopper],
            "postpone_start": True,
        }
        run.append(model.ShiftAmountActivity(**shift_amount_loading_data))
         

        sequential_activity_data = {
            "env": my_env,
            "name": "run",
            "registry": registry,
            "sub_processes": run,
            "postpone_start": True,
        }
        sequential_activity = model.SequentialActivity(**sequential_activity_data)
        
        while_data = {
            "env": my_env,  # The simpy environment defined in the first cel
            "name": "run while",  # We are moving soil
            "registry": registry,
            "sub_process": sequential_activity,
            "condition_event": [{"type":"container", "concept": from_site, "state":"empty"}],
            "postpone_start": False,
        }
        run_activity = model.WhileActivity(**while_data)

while Activity keep_resources []
start put_available
register new event 
start event instance None
while Activity keep_resources []
start put_available
check put_available events
register new event 
start event instance None
while Activity keep_resources []
start put_available
check put_available events
register new event 
start event instance None
while Activity keep_resources []
start put_available
check put_available events
register new event 
start event instance None
while Activity keep_resources []
start put_available
check put_available events
register new event 
start event instance None
while Activity keep_resources []
start put_available
check put_available events
register new event 
start event instance None


In [5]:
my_env.run()

conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
put_callback - id_ default
{}
put_callback - id_ default
{}
put_callback - id_ default
{}
put_callback - id_ default
{}
put_callback - id_ default
{}
put_callback - id_ default
{}
put_callback - id_ default
{}
Mover_move after mover resource request
Mover_

RuntimeError: Attempting to shift content from an empty origin or to a full destination. ({'origin.41cea906-aaff-11ea-aa1f-2079183a5c9a': 18, 'destination.41cef59c-aaff-11ea-9173-2079183a5c9a': 0})

In [6]:
objects = [cutter1, cutter2, hopper1, hopper2, hopper3, from_site]

activities = []
for obj in objects:
    print(set(obj.log["Message"]))
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

            
C = np.linspace(0,255, len(activities))
colors = {}

for i in range(len(activities)):
    colors[i] = f'rgb({int(C[i]/2)},{int(C[len(C) - 1 - i]/1.5)},{int(C[i])})'

vessel_planning(objects, activities, colors)

{'transfer default to Hopper1', 'transfer default to Hopper2', 'transfer default to Hopper3'}
{'transfer default to Hopper1', 'transfer default to Hopper2', 'transfer default to Hopper3'}
{'transfer default to Hopper1', 'transfer default to Dumplocatie', 'sailing filled', 'sailing empty'}
{'transfer default to Hopper2', 'sailing filled', 'sailing empty'}
{'transfer default to Hopper3', 'sailing filled', 'sailing empty'}
{'transfer default to Hopper1', 'transfer default to Hopper2', 'transfer default to Hopper3'}


In [9]:
from_site.container.get_level()

18

In [10]:
to_site.container.get_level()

1